In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
%cd gdrive/MyDrive/HW12

/content/gdrive/MyDrive/HW12


In [3]:
!ls

In [4]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip uninstall imgaug -y
!pip install imgaug==0.2.5 

Found existing installation: imgaug 0.2.7
Uninstalling imgaug-0.2.7:
  Successfully uninstalled imgaug-0.2.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 562 kB 7.4 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=2a60af1f38731a16d0d03225c968eb34002e60b6e51dbe7fea11348b84ac6d66
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug


In [13]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.
import os

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.10.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.10.0) to /tmp/pip-install-ysbgrx1y/nemo-toolkit_5f470f7b9d1c486eb03dccc63b4b6549
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-i

In [27]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
!pip install folium==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.9 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=0553627d672fb2773d7c2de20fdb1f2ee7c6870931dbebb1c3ec2cc461c2c14c
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [28]:
# In a conda environment, you would use the following command
# Update Numba to > 0.53
# conda install -c conda-forge numba
# or
# conda update -c conda-forge numba

# For pip based environments,
# Update Numba to > 0.54
!pip install --upgrade numba==0.54.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Intro to Transducers

By following the earlier tutorials for Automatic Speech Recognition in NeMo, one would have probably noticed that we always end up using [Connectionist Temporal Classification (CTC) loss](https://distill.pub/2017/ctc/) in order to train the model. Speech Recognition can be formulated in many different ways, and CTC is a more popular approach because it is a monotonic loss - an acoustic feature at timestep $t_1$ and $t_2$ will correspond to a target token at timestep $u_1$ and only then $u_2$. This monotonic property significantly simplifies the training of ASR models and speeds up convergence. However, it has certain drawbacks that we will discuss below.

In general, ASR can be described as a sequence-to-sequence prediction task - the original sequence is an audio sequence (often transformed into mel spectrograms). The target sequence is a sequence of characters (or subword tokens). Attention models are capable of the same sequence-to-sequence prediction tasks. They can even perform better than CTC due to their autoregressive decoding. However, they lack certain inductive biases that can be leveraged to stabilize and speed up training (such as the monotonicity exhibited by the CTC loss). Furthermore, by design, attention models require the entire sequence to be available to align the sequence to the output, thereby preventing their use for streaming inference.

Then comes the [Transducer Loss](https://arxiv.org/abs/1211.3711). Proposed by Alex Graves, it aimed to resolve the issues in CTC loss while resolving the transcription accuracy issues by performing autoregressive decoding. 



## Drawbacks of Connectionist Temporal Classification (CTC)

CTC is an excellent loss to train ASR models in a stable manner but comes with certain limitations on model design. If we presume speech recognition to be a sequence-to-sequence problem, let $T$ be the sequence length of the acoustic model's output, and let $U$ be the sequence length of the target text transcript (post tokenization, either as characters or subwords). 

-------

1) CTC imposes the limitation : $T \ge U$. Normally, this assumption is naturally valid because $T$ is generally a lot longer than the final text transcription. However, there are many cases where this assumption fails.

- Acoustic model performs downsampling to such a degree that $T \ge U$. Why would we want to perform so much downsampling? For convolutions, longer sequences take more stride steps and more memory. For Attention-based models (say Conformer), there's a quadratic memory cost of computing the attention step in proportion to $T$. So more downsampling significantly helps relieve the memory requirements. There are ways to bypass this limitation, as discussed in the `ASR_with_Subword_Tokenization` notebook, but even that has limits.

- The target sequence is generally very long. Think of languages such as German, which have very long translations for short English words. In the task of ASR, if there is more than 2x downsampling and character tokenization is used, the model will often fail to learn due to this CTC limitation.

2) Tokens predicted by models which are trained with just CTC loss are assumed to be *conditionally independent*. This means that, unlike language models where *h*-*e*-*l*-*l* as input would probably predict *o* to complete *hello*, for CTC trained models - any character from the English alphabet has equal likelihood for prediction. So CTC trained models often have misspellings or missing tokens when transcribing the audio segment to text. 

- Since we often use the Word Error Rate (WER) metric when evaluating models, even a single misspelling contributes significantly to the "word" being incorrect. 

- To alleviate this issue, we have to resort to Beam Search via an external language model. While this often works and significantly improves transcription accuracy, it is a slow process and involves large N-gram or Neural language models. 

--------

Let's see CTC loss's limitation (1) in action:

In [29]:
import torch
import torch.nn as nn

In [30]:
T = 10  # acoustic sequence length
U = 16  # target sequence length
V = 28  # vocabulary size

def get_sample(T, U, V, require_grad=True):
  torch.manual_seed(0)

  acoustic_seq = torch.randn(1, T, V + 1, requires_grad=require_grad)
  acoustic_seq_len = torch.tensor([T], dtype=torch.int32)  # actual seq length in padded tensor (here no padding is done)

  target_seq = torch.randint(low=0, high=V, size=(1, U))
  target_seq_len = torch.tensor([U], dtype=torch.int32)

  return acoustic_seq, acoustic_seq_len, target_seq, target_seq_len

In [31]:
# First, we use CTC loss in the general sense.
loss = torch.nn.CTCLoss(blank=V, zero_infinity=False)

acoustic_seq, acoustic_seq_len, target_seq, target_seq_len = get_sample(T, U, V)

# CTC loss expects acoustic sequence to be in shape (T, B, V)
val = loss(acoustic_seq.transpose(1, 0), target_seq, acoustic_seq_len, target_seq_len)
print("CTC Loss :", val)

val.backward()
print("Grad of Acoustic model (over V):", acoustic_seq.grad[0, 0, :])

CTC Loss : tensor(inf, grad_fn=<MeanBackward0>)
Grad of Acoustic model (over V): tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan])


In [32]:
# Next, we use CTC loss with `zero_infinity` flag set.
loss = torch.nn.CTCLoss(blank=V, zero_infinity=True)

acoustic_seq, acoustic_seq_len, target_seq, target_seq_len = get_sample(T, U, V)

# CTC loss expects acoustic sequence to be in shape (T, B, V)
val = loss(acoustic_seq.transpose(1, 0), target_seq, acoustic_seq_len, target_seq_len)
print("CTC Loss :", val)

val.backward()
print("Grad of Acoustic model (over V):", acoustic_seq.grad[0, 0, :])

CTC Loss : tensor(0., grad_fn=<MeanBackward0>)
Grad of Acoustic model (over V): tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.])


-------

As we saw, CTC loss in general case will not be able to compute the loss or the gradient when $T \ge U$. In the PyTorch specific implementation of CTC Loss, we can specify a flag `zero_infinity`, which explicitly checks for such cases, zeroes out the loss and the gradient if such a case occurs. The flag allows us to train a batch of samples where some samples may accidentally violate this limitation, but training will not halt, and gradients will not become NAN.

## What is the Transducer Loss ?

![](https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/images/transducer.png?raw=true)

A model that seeks to use the Transducer loss is composed of three models that interact with each other. They are:

-------

1) **Acoustic model** : This is nearly the same acoustic model used for CTC models. The output shape of these models is generally $(Batch, \, T, \, AM-Hidden)$. You will note that unlike for CTC, the output of the acoustic model is no longer passed through a decoder layer which would have the shape  $(Batch, \, T, \, Vocabulary + 1)$.

2) **Prediction / Decoder model** : The prediction model accepts a sequence of target tokens (in the case of ASR, text tokens) and is usually a causal auto-regressive model that is tasked with predicting some hidden feature dimension of shape $(Batch, \, U, \, Pred-Hidden)$.

3) **Joint model** : This model accepts the outputs of the Acoustic model and the Prediction model and joins them to compute a joint probability distribution over the vocabulary space to compute the alignments from Acoustic sequence to Target sequence. The output of this model is of the shape $(Batch, \, T, \, U, \, Vocabulary + 1)$.

--------

During training, the transducer loss is computed on the output of the joint model, which computes the joint probability distribution of a target vocabulary token $v_{t, u}$ (for all $v \in V$) being predicted given the acoustic feature at timestep $t \le T$ and the prediction network features at timestep $u \le U$.

--------

During inference, we perform a single forward pass over the Acoustic Network to obtain the features of shape $(Batch, \, T, \, AM-Hidden)$, and autoregressively perform the forward passes of the Prediction Network and the Joint Network to decode several $u \le U$ target tokens per acoustic timestep $t \le T$. We will discuss decoding in the following sections.


---------

**Note**: For an excellent in-depth explanation of how Transducer loss works, how it computes the alignment, and how the gradient of this alignment is calculated, we highly encourage you to read this post about [Sequence-to-sequence learning with Transducers by Loren Lugosch](https://lorenlugosch.github.io/posts/2020/11/transducer/).

---------

## Benefits of Transducer Loss

Now that we understand what a Transducer model is comprised of and how it is trained, the next question that comes to mind is - What is the benefit of the Transducer loss?

------

1) It is a monotonic loss (similar to CTC). Monotonicity speeds up convergence and does not require auxiliary losses to stabilize training (which is required when using only attention-based loss for sequence-to-sequence training).

2) Autoregressive decoding enables the model to implicitly have a dependency between predicted tokens (the conditional independence assumption of CTC trained models is corrected). As such, missing characters or incorrect spellings are less frequent (but still exist since no model is perfect).

3) It no longer has the $T \ge U$ limitation that CTC imposed. This is because the total joint probability distribution is calculated now - mapping every acoustic timestep $t \le T$ to one or more target timestep $u \le U$. This means that for each timestep $t$, the model has at most $U$ tokens that it can predict, and therefore in the extreme case, it can predict a total of $T \times U$ tokens!

## Drawbacks of Transducer Loss

All of these benefits come with certain costs. As is (almost) always the case in machine learning, there is no free lunch. 

-------

1) During training, the Joint model is required to compute a joint matrix of shape $(Batch, \, T, \, U, \, Vocabulary + 1)$. If you consider the value of these constants for a general dataset like Librispeech, $T \sim 1600$, $U \sim 450$ (with character encoding) and vocabulary $V \sim 28+1$. Considering a batch size of 32, that total memory cost comes out to roughly **2.7 GB** at float precision. The model would also need another **2.7 GB** for the gradients. Of course, the model needs more memory still for the actual Acoustic model + Prediction model + their gradients. Note, however - this issue can be *partially* resolved with some simple tricks, which are discussed in the next tutorial. Also, this memory cost is no longer an issue during inference!

2) Autoregressive decoding is slow. Much slower than CTC models, which require just a simple argmax of the output tensor. So while we do get superior transcription quality, we sacrifice decoding speed.

--------

Let's check that RNNT loss no longer shows the limitations of CTC loss - 

In [33]:
T = 10  # acoustic sequence length
U = 16  # target sequence length
V = 28  # vocabulary size

def get_rnnt_sample(T, U, V, require_grad=True):
  torch.manual_seed(0)

  joint_tensor = torch.randn(1, T, U + 1, V + 1, requires_grad=require_grad)
  acoustic_seq_len = torch.tensor([T], dtype=torch.int32)  # actual seq length in padded tensor (here no padding is done)

  target_seq = torch.randint(low=0, high=V, size=(1, U))
  target_seq_len = torch.tensor([U], dtype=torch.int32)

  return joint_tensor, acoustic_seq_len, target_seq, target_seq_len

In [34]:
import nemo.collections.asr as nemo_asr

[NeMo W 2022-07-29 06:40:41 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [35]:
joint_tensor, acoustic_seq_len, target_seq, target_seq_len = get_rnnt_sample(T, U, V)

# RNNT loss expects joint tensor to be in shape (B, T, U, V)
loss = nemo_asr.losses.rnnt.RNNTLoss(num_classes=V)

# Uncomment to check out the keyword arguments required to call the RNNT loss
print("Transducer loss input types :", loss.input_types)
print()

val = loss(log_probs=joint_tensor, targets=target_seq, input_lengths=acoustic_seq_len, target_lengths=target_seq_len)
print("Transducer Loss :", val)

val.backward()
print("Grad of Acoustic model (over V):", joint_tensor.grad[0, 0, 0, :])

Transducer loss input types : {'log_probs': NeuralType(axis=(batch, time, time, dimension), element_type=LogprobsType), 'targets': NeuralType(axis=(batch, time), element_type=LabelsType), 'input_lengths': NeuralType(axis=(batch,), element_type=LengthsType), 'target_lengths': NeuralType(axis=(batch,), element_type=LengthsType)}

Transducer Loss : tensor(79.8858, grad_fn=<MeanBackward0>)
Grad of Acoustic model (over V): tensor([ 0.0077,  0.0075,  0.0184,  0.0153,  0.0551, -0.7121,  0.0172,  0.0028,
         0.0326,  0.0067,  0.0335,  0.0321,  0.0266,  0.0814,  0.0721,  0.0184,
         0.0061,  0.0043,  0.0416,  0.0522,  0.0430,  0.0050,  0.0168,  0.1506,
         0.0500,  0.0131,  0.0198,  0.0284, -0.1461])


# Configure a Transducer Model

We now understand a bit more about the transducer loss. Next, we will take a deep dive into how to set up the config for a transducer model.

Transducer configs contain a fair bit more detail compared to CTC configs. However, the vast majority of the defaults can be copied and pasted into your configs to have a perfectly functioning transducer model!

------

Let us download one of the transducer configs already available in NeMo to analyze the components.

In [36]:
import os

if not os.path.exists("contextnet_rnnt.yaml"):
  !wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/contextnet_rnnt/contextnet_rnnt.yaml

--2022-07-29 06:41:09--  https://raw.githubusercontent.com/NVIDIA/NeMo/r1.10.0/examples/asr/conf/contextnet_rnnt/contextnet_rnnt.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18863 (18K) [text/plain]
Saving to: ‘contextnet_rnnt.yaml’

contextnet_rnnt.yam 100%[===================>]  18.42K  --.-KB/s    in 0.004s  

2022-07-29 06:41:09 (4.64 MB/s) - ‘contextnet_rnnt.yaml’ saved [18863/18863]



In [37]:
from omegaconf import OmegaConf, open_dict

cfg = OmegaConf.load('contextnet_rnnt.yaml')

## Model Defaults

Since the transducer model is comprised of three separate models working in unison, it is practical to have some shared section of the config. That shared section is called `model.model_defaults`.

In [38]:
print(OmegaConf.to_yaml(cfg.model.model_defaults))

filters: 1024
repeat: 5
dropout: 0.1
separable: true
se: true
se_context_size: -1
kernel_size_factor: 1.0
enc_hidden: 640
pred_hidden: 640
joint_hidden: 640



-------

Of the many components shared here, the last three values are the primary components that a transducer model **must** possess. They are :

1) `enc_hidden`: The hidden dimension of the final layer of the Encoder network.

2) `pred_hidden`: The hidden dimension of the final layer of the Prediction network.

3) `joint_hidden`: The hidden dimension of the intermediate layer of the Joint network.

--------

One can access these values inside the config by using OmegaConf interpolation as follows :

```yaml
model:
  ...
  decoder:
    ...
    prednet:
      pred_hidden: ${model.model_defaults.pred_hidden}
```

## Acoustic Model

As we discussed before, the transducer model is comprised of three models combined. One of these models is the Acoustic (encoder) model. We should be able to drop in any CTC Acoustic model config into this section of the transducer config.

The only condition that needs to be met is that **the final layer of the acoustic model must have the dimension defined in `model_defaults.enc_hidden`**.

## Decoder / Prediction Model

The Prediction model is generally an autoregressive, causal model that consumes text tokens and returns embeddings that will be used by the Joint model. 

**This config can be dropped into any custom transducer model with no modification.**

In [39]:
print(OmegaConf.to_yaml(cfg.model.decoder))

_target_: nemo.collections.asr.modules.RNNTDecoder
normalization_mode: null
random_state_sampling: false
blank_as_pad: true
prednet:
  pred_hidden: ${model.model_defaults.pred_hidden}
  pred_rnn_layers: 1
  t_max: null
  dropout: 0.1



------

This config will build an LSTM based Transducer Decoder model. Let us discuss some of the important arguments:

1) `blank_as_pad`: In ordinary transducer models, the embedding matrix does not acknowledge the `Transducer Blank` token (similar to CTC Blank). However, this causes the autoregressive loop to be more complicated and less efficient. Instead, this flag which is set by default, will add the `Transducer Blank` token to the embedding matrix - and use it as a pad value (zeros tensor). This enables more efficient inference without harming training.

2) `prednet.pred_hidden`: The hidden dimension of the LSTM and the output dimension of the Prediction network.


## Joint Model

The Joint model is a simple feed-forward Multi-Layer Perceptron network. This MLP accepts the output of the Acoustic and Prediction models and computes a joint probability distribution over the entire vocabulary space.

**This config can be dropped into any custom transducer model with no modification.**

In [40]:
print(OmegaConf.to_yaml(cfg.model.joint))

_target_: nemo.collections.asr.modules.RNNTJoint
log_softmax: null
preserve_memory: false
fuse_loss_wer: true
fused_batch_size: 16
jointnet:
  joint_hidden: ${model.model_defaults.joint_hidden}
  activation: relu
  dropout: 0.1



------

The Joint model config has several essential components which we discuss below :

1) `log_softmax`: Due to the cost of computing softmax on such large tensors, the Numba CUDA implementation of RNNT loss will implicitly compute the log softmax when called (so its inputs should be logits). The CPU version of the loss doesn't face such memory issues so it requires log-probabilities instead. Since the behaviour is different for CPU-GPU, the `None` value will automatically switch behaviour dependent on whether the input tensor is on a CPU or GPU device.

2) `preserve_memory`: This flag will call `torch.cuda.empty_cache()` at certain critical sections when computing the Joint tensor. While this operation might allow us to preserve some memory, the empty_cache() operation is tremendously slow and will slow down training by an order of magnitude or more. It is available to use but not recommended.

3) `fuse_loss_wer`: This flag performs "batch splitting" and then "fused loss + metric" calculation. It will be discussed in detail in the next tutorial that will train a Transducer model.

4) `fused_batch_size`: When the above flag is set to True, the model will have two distinct "batch sizes". The batch size provided in the three data loader configs (`model.*_ds.batch_size`) will now be the `Acoustic model` batch size, whereas the `fused_batch_size` will be the batch size of the `Prediction model`, the `Joint model`, the `transducer loss` module and the `decoding` module.

5) `jointnet.joint_hidden`: The hidden intermediate dimension of the joint network.

## Transducer Decoding

Models which have been trained with CTC can transcribe text simply by performing a regular argmax over the output of their decoder.

For transducer-based models, the three networks must operate in a synchronized manner in order to transcribe the acoustic features.

The following section of the config describes how to change the decoding logic of the transducer model.


**This config can be dropped into any custom transducer model with no modification.**

In [41]:
print(OmegaConf.to_yaml(cfg.model.decoding))

strategy: greedy_batch
greedy:
  max_symbols: 10
beam:
  beam_size: 4
  score_norm: true
  return_best_hypothesis: false
  softmax_temperature: 1.0
  tsd_max_sym_exp: 10
  alsd_max_target_len: 5.0
  maes_num_steps: 2
  maes_prefix_alpha: 1
  maes_expansion_beta: 2
  maes_expansion_gamma: 2.3



-------

The most important component at the top level is the `strategy`. It can take one of many values:

1) `greedy`: This is sample-level greedy decoding. It is generally exceptionally slow as each sample in the batch will be decoded independently. For publications, this should be used alongside batch size of 1 for exact results.

2) `greedy_batch`: This is the general default and should nearly match the `greedy` decoding scores (if the acoustic features are not affected by feature mixing in batch mode). Even for small batch sizes, this strategy is significantly faster than `greedy`.

3) `beam`: Runs beam search with the implicit language model of the Prediction model. It will generally be quite slow, and might need some tuning of the beam size to get better transcriptions.

4) `tsd`: Time synchronous decoding. Please refer to the paper:                 [Alignment-Length Synchronous Decoding for RNN Transducer](https://ieeexplore.ieee.org/document/9053040) for details on the algorithm implemented. Time synchronous decoding (TSD) execution time grows by the factor T * max_symmetric_expansions. For longer sequences, T is greater and can therefore take a long time for beams to obtain good results. TSD also requires more memory to execute.

5) `alsd`: Alignment-length synchronous decoding. Please refer to the paper: [Alignment-Length Synchronous Decoding for RNN Transducer](https://ieeexplore.ieee.org/document/9053040) for details on the algorithm implemented. Alignment-length synchronous decoding (ALSD) execution time is faster than TSD, with a growth factor of T + U_max, where U_max is the maximum target length expected during execution. Generally, T + U_max < T * max_symmetric_expansions. However, ALSD beams are non-unique. Therefore it is required to use larger beam sizes to achieve the same (or close to the same) decoding accuracy as TSD. For a given decoding accuracy, it is possible to attain faster decoding via ALSD than TSD.

-------

Below, we discuss the various decoding strategies.

### Greedy Decoding

When `strategy` is one of `greedy` or `greedy_batch`, an additional subconfig of `decoding.greedy` can be used to set an important decoding value.

In [42]:
print(OmegaConf.to_yaml(cfg.model.decoding.greedy))

max_symbols: 10



-------

This argument `max_symbols` is the maximum number of `target token` decoding steps $u \le U$ per acoustic timestep $t \le T$. Note that during training, this was implicitly constrained by the shape of the joint matrix (max_symbols = $U$). However, there is no such $U$ upper bound during inference (we don't have the ground truth $U$).

So we explicitly set a heuristic upper bound on how many decoding steps can be performed per acoustic timestep. Generally a value of 5 and above is sufficient.

### Beam Decoding

Next, we discuss the subconfig when `strategy` is one of `beam`, `tsd` or `alsd`.

In [43]:
print(OmegaConf.to_yaml(cfg.model.decoding.beam))

beam_size: 4
score_norm: true
return_best_hypothesis: false
softmax_temperature: 1.0
tsd_max_sym_exp: 10
alsd_max_target_len: 5.0
maes_num_steps: 2
maes_prefix_alpha: 1
maes_expansion_beta: 2
maes_expansion_gamma: 2.3



------

There are several important arguments in this section :

1) `beam_size`: This determines the beam size for all types of beam decoding strategy. Since this is implemented in PyTorch, large beam sizes will take exorbitant amounts of time.

2) `score_norm`: Whether to normalize scores prior to pruning the beam.

3) `return_best_hypothesis`: If beam search is being performed, we can choose to return just the best hypothesis or all the hypotheses.

4) `tsd_max_sym_exp`: The maximum symmetric expansions allowed per timestep during beam search. Larger values should be used to attempt decoding of longer sequences, but this in turn increases execution time and memory usage.

5) `alsd_max_target_len`: The maximum expected target sequence length during beam search. Larger values allow decoding of longer sequences at the expense of execution time and memory.


## Transducer Loss

Finally, we reach the Transducer loss config itself. This section configures the type of Transducer loss itself, along with possible sub-sections.

**This config can be dropped into any custom transducer model with no modification.**

In [44]:
print(OmegaConf.to_yaml(cfg.model.loss))

loss_name: default
warprnnt_numba_kwargs:
  fastemit_lambda: 0.001
  clamp: -1.0



---------

The loss config is based on a resolver pattern and can be used as follows:

1) `loss_name`: `default` is generally a good option. Will select one of the available resolved losses and match the kwargs from a sub-configs passed via explicit `{loss_name}_kwargs` sub-config. 

2) `{loss_name}_kwargs`: This sub-config is passed to the resolved loss above and can be used to configure the resolved loss.

### WarpRNNT Numba Loss

The default transducer loss implemented in NeMo is a Numba port of the excellent CUDA implementation of Transducer Loss found in https://github.com/HawkAaron/warp-transducer.

It should suffice for most use cases (CPU / GPU) transducer training.

### FastEmit Regularization

Recently proposed regularization approach - [FastEmit: Low-latency Streaming ASR with Sequence-level Emission Regularization](https://arxiv.org/abs/2010.11148) allows us near-direct control over the latency of transducer models.

Refer to the above paper for results and recommendations of `fastemit_lambda`.

# Next Steps

After that deep dive into how to configure Transducer models, the next tutorial will use one such config to build a transducer model and train it on a small dataset. We will then move on to exploring various decoding strategies and how to evaluate the model.